In [2]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')



In [3]:
from config_paper import key_id, secret_key
from cross_signal import *

In [4]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(key_id, secret_key,base_url )

In [5]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-04-26'

In [6]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['ema_high'] = tab.EMA(dataframe, timeperiod=10, price='high')
        dataframe['ema_close'] = tab.EMA(dataframe, timeperiod=10, price='close')
        dataframe['ema_low'] = tab.EMA(dataframe, timeperiod=10, price='low')
        
        
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        dataframe['ultosc'] = ta.ULTOSC(dataframe['high'],dataframe['low'],dataframe['close'])
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        stoch_fast = tab.STOCHF(dataframe, 10.0, 3.0, 0.0, 3.0, 0.0)
        dataframe['fastd'] = stoch_fast['fastd']
        dataframe['fastk'] = stoch_fast['fastk']
        dataframe['adx'] = tab.ADX(dataframe)
        dataframe['cci'] = tab.CCI(dataframe)
        # required for graphing

        return dataframe

In [7]:
account = api.get_account()
print(account.status)
account.buying_power

ACTIVE


'400000'

In [8]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [42]:
api = tradeapi.REST(key_id ,secret_key )

barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.25

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)


In [39]:
returneddata = api.get_barset('FLS',barTimeframe,limit=150).df
returneddata2=api.polygon.historic_agg_v2( 'DISCA', 1, 'minute',_from='2020-04-23', to=startdate,).df
#data_indic=populateindicators(returneddata['AMP'])


In [40]:
returneddata2

,open,high,low,close,volume
timestamp,,,,,
2020-04-23 05:12:00-04:00,20.2400,20.24,20.24,20.24,150
2020-04-23 05:13:00-04:00,20.2400,20.24,20.24,20.24,403
2020-04-23 06:40:00-04:00,20.1000,20.10,20.05,20.05,1629
2020-04-23 06:41:00-04:00,20.0000,20.00,20.00,20.00,100
2020-04-23 06:44:00-04:00,20.0500,20.05,20.05,20.05,274
...,...,...,...,...,...
2020-04-24 15:59:00-04:00,21.0034,21.02,20.98,21.01,64325
2020-04-24 16:00:00-04:00,21.0200,21.02,21.02,21.02,60641
2020-04-24 16:40:00-04:00,21.6000,21.60,21.60,21.60,1000


In [44]:
for minu in range(1) :
        
    for i in range(assetListLen) :
        try:
            symbol = assetsToTrade[i]

            returned_data = api.polygon.historic_agg_v2(symbol, 1, 'minute',_from='2020-04-20', to=startdate,).df
            #api.get_barset(symbol,barTimeframe,limit=150).df


            # Calculated trading indicators
            #populate exponential moving averages
            data=populateindicators(returned_data) 


             # Create the buy signal here
            if data['adx'][-1] > 22 and \
            data['mfi'][-1] < 25 and \
            data['ultosc'][-1] < 25 and \
            data['rsi'][-1] < 33 and \
            data['minusdi'][-1] > 25 and \
            data['minusdi'][-1] > data['plusdi'][-1]:


                openPosition = api.get_position(symbol)

                # Opens new position if one does not exist
                if openPosition == 0:
                    cashBalance = api.get_account().cash

                    targetPositionSize = cashBalance / (price / positionSizing) # Calculates required position size

                    returned = api.submit_order(symbol,targetPositionSize,"buy","market","day") # Market order to open position
                    print(returned)


            elif data['adx'][-1] > 22 and \
            data['mfi'][-1] > 63 and \
            data['rsi'][-1] > 75 and \
            data['plusdi'][-1] > 25 and \
            data['minusdi'][-1] < data['plusdi'][-1]:
                # Closes position  SMA50
                openPosition = api.get_position(symbol)

                returned = api.submit_order(symbol,openPosition,"sell","market","day") # Market order to fully close position
                print(returned)
        
        except:
                print(symbol)
                
        #interval should be an integer, the number of seconds to wait
    time.sleep(20)

AMP
BRKB
BFB
FLS
HPQ
MKTK
NLOK
RTN
UTX

